# *uxtbpy* tutorial

This notebook goes through the basic usage *uxtbpy* and is organized into four parts: 
1. Requirements
2. XtbRunner
3. StdaRunner
4. Limitations

### Requirements

For *uxtbpy* to function properly you need working installations of [*xTB*](https://github.com/grimme-lab/xtb), [*xTB4sTDA*](https://github.com/grimme-lab/xtb4stda) and [*sTDA*](https://github.com/grimme-lab/std2). These can be installed from the corresponding GitHub pages and should be accessible on your system via *xtb*, *xtb4stda* and *stda*, respectively.

If you are on a Unix system you can use the below *bash* script to install the necessary binaries.

```bash
#!/bin/bash -l

# download binaries
wget https://github.com/grimme-lab/xtb/releases/download/v6.7.1/xtb-6.7.1-linux-x86_64.tar.xz
wget https://github.com/grimme-lab/stda/releases/download/v1.6.3/xtb4stda
wget https://github.com/grimme-lab/stda/releases/download/v1.6.3/stda_v1.6.3

# extract xtb
tar -xvf xtb-6.7.1-linux-x86_64.tar.xz
rm xtb-6.7.1-linux-x86_64.tar.xz

# rename stda
mv stda_v1.6.3 stda

# add execution permissions
chmod +x xtb-dist/bin/xtb
chmod +x xtb4stda
chmod +x stda

# move binaries to /usr/local/bin/
mv xtb-dist/bin/xtb /usr/local/bin/
mv xtb4stda /usr/local/bin/
mv stda /usr/local/bin/

# clean up
rm -r xtb-dist/
```


*uxtbpy* does not require any external packages and can be directly installed from GitHub using *pip*.

In [ ]:
!pip install git+https://github.com/hkneiding/uxtbpy

After installation we can import the package and decide on a working directory from which *xTB* and *sTDA* will be launched.

In [ ]:
import uxtbpy

working_directory = "./wd/"

### XtbRunner

The *XtbRunner* module interfaces the *xTB* binary (*xtb*) for conveniently running jobs (e.g. singlepoints and optimizations) directly from *Python*. After instantiating a *XtbRunner* object, passing to it as argument our chosen working directory path, we can directly run jobs providing a path to molecule file and a list of parameters. If no working directory path is provided, the constructor will default to "./.temp/". In the example below, we request an optimization with the overall molecular charge set to 0. For the available options please refer to the [xTB GitHub pages and documentation](https://github.com/grimme-lab/xtb).

The run call will return a parsed dictionary of properties found in the *xTB* output. Note that depending on the parameters appended to the call (e.g. *--opt* vs *--ohess*) different properties will be calculated by *xTB* and therefore also different properties (keys) will be present in the resulting dictionary.

In [ ]:
xtb_runner = uxtbpy.XtbRunner(working_directory=working_directory)
result = xtb_runner.run_from_file("cisplatin.xyz", parameters=["--opt", "--chrg 0"])

print(result.keys())

### StdaRunner

In a similar fashion, the *StdaRunner* module interfaces the *xTB4sTDA* (*xtb4stda*) and *sTDA* (*stda*) binaries for conveniently running jobs to (e.g. excited state calculations) directly from *Python*. In order to run *sTDA* jobs based on *xTB* we need the *xTB4sTDA* program to generate an appropriate wavefunction file that the *sTDA* program can utilize. The *StdaRunner* takes care of this two-step process and is setup in the same way as the *XtbRunner* by providing a path to a chosen working directory. Jobs can also be started in the same way by providing a path to a molecule file as well as a list of *xTB4sTDA* and a list of *sTDA* parameters. In the example below, we utilize the optimized geometry generated by our previous *xTB* job found in our working directory. For the available options please refer to the [xTB4sTDA GitHub pages](https://github.com/grimme-lab/xtb4stda) and [sTDA GitHub pages](https://github.com/grimme-lab/std2).

The run call will return a parsed dictionary of properties found in the *sTDA* output.

In [ ]:
stda_runner = uxtbpy.StdaRunner(working_directory=working_directory)
result = stda_runner.run_from_file(working_directory + "xtbopt.xyz", xtb4stda_parameters=["-chrg 0"], stda_parameters=[])

print(result.keys())

### Limitations

*uxtbpy*, in particular the parsing modules (*XtbOutputParser*, *StdaOutputParser*), are developed with respect to certain *xTB* and *sTDA* versions. While the calling of their binaries will be largely unaffected in case of changes or updates to the programs, the parsers might fail to capture all relevant data if the structure of the program output changes. Therefore, *uxtbpy* is configured to also write the STDOUT of all exectued binaries to file in the corresponding working directory. This also allows for custom processing of the output, in case you require information that is not captured by the parsers.

In [ ]:
with open(working_directory + "xtb.stdout") as fh:
    xtb_stdout = fh.read()
with open(working_directory + "xtb4stda.stdout") as fh:
    xtb4stda_stdout = fh.read()
with open(working_directory + "stda.stdout") as fh:
    stda_stdout = fh.read()